In [1]:
import llama
import torch 
MODEL = "openchat/openchat_3.5"

LOAD_8BIT = False # True

tokenizer = llama.LlamaTokenizer.from_pretrained(MODEL)
model = llama.LlamaForCausalLM.from_pretrained(MODEL, low_cpu_mem_usage=True, load_in_8bit=LOAD_8BIT, torch_dtype=torch.float16, device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using a model of type mistral to instantiate a model of type llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/paul/miniconda3/envs/iti/lib/python3.8/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/paul/miniconda3/envs/iti/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [1]:
from accelerate.utils import BnbQuantizationConfig

bnb_quantization_config = BnbQuantizationConfig(load_in_8bit=True, llm_int8_threshold = 6)

from accelerate.utils import load_and_quantize_model
quantized_model = load_and_quantize_model(empty_model, weights_location=weights_location, bnb_quantization_config=bnb_quantization_config, device_map = "auto")

In [15]:
i = tokenizer(["hi"], return_tensors="pt")

In [18]:
i['input_ids'].to("cuda")

tensor([[    1, 12014]], device='cuda:0')

In [22]:
embeds = model.model.embed_tokens(i['input_ids'].to("cuda"))#.shape

In [24]:
model.model.layers[0].self_attn

LlamaAttention(
  (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
  (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
  (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
  (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
  (rotary_emb): LlamaRotaryEmbedding()
  (att_out): Identity()
  (value_out): Identity()
  (head_out): Identity()
)

In [23]:
model.model.layers[0].self_attn(embeds)

(tensor([[[ 1.8179e-05, -2.2531e-05, -4.4465e-05,  ...,  7.2598e-05,
            2.6822e-06,  1.3590e-05],
          [ 1.8179e-05, -2.2531e-05, -4.4465e-05,  ...,  7.2598e-05,
            2.6822e-06,  1.3590e-05]]], device='cuda:0', dtype=torch.float16,
        grad_fn=<MatMul8bitLtBackward>),
 None,
 None)

In [6]:
model.model.layers[0].self_attn.head_out

Identity()

In [9]:
import torch


def get_module(model, name):
    """
    Finds the named module within the given model.
    """
    for n, m in model.named_modules():
        if n == name:
            return m
    raise LookupError(name)


module = model
layer = 'model.layers.31.self_attn.head_out'
get_module(module, layer)

Identity()